In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os
import pickle

Using TensorFlow backend.


In [2]:
try:
    with open('X_tundra.pkl', 'rb') as handle:
        X = pickle.load(handle)
except:
    path = './unzipped_images/tundra/'
    X = []
    for filename in os.listdir(path):
        X.append(img_to_array(load_img(path + filename))/255)
    X = np.array(X)
    with open('X_tundra.pkl', 'wb') as handle:
        pickle.dump(X, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
X.shape

(5000, 256, 256, 3)

In [4]:
Xtrain = X[:4500]

In [5]:
Xval = X[4500:]

In [6]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

batch_size = 10
def train_generator(batch_size):
    for batch in train_datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0] / 100 # scale to [0, 1] bc neural networks prefer small input values
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [7]:
val_datagen = ImageDataGenerator()

def val_generator(batch_size):
    for batch in val_datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0] / 100 # scale to [0, 1] bc neural networks prefer small input values
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)